## Stage 1: Installing dependencies and setting up the environment

In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 35.8MB/s 
     |████████████████████████████████| 368kB 45.3MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


## Stage 2: Importing project dependencies

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.1'

## Stage 3: Dataset preprocessing

### Loading the FashionMNIST dataset

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Image normalization

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Reshaping of the dataset

In [5]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Stage 4: Building a model

### Defining the model

In [0]:
model = tf.keras.models.Sequential()

### Building the model

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compiling the model

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Training the model

In [11]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.5339 - sparse_categorical_accuracy: 0.8106
Epoch 2/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3992 - sparse_categorical_accuracy: 0.8561
Epoch 3/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3679 - sparse_categorical_accuracy: 0.8658
Epoch 4/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3447 - sparse_categorical_accuracy: 0.8738
Epoch 5/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3290 - sparse_categorical_accuracy: 0.8791


### Evaluating the model

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 51us/sample - loss: 0.3536 - sparse_categorical_accuracy: 0.8739
Test accuracy: 0.8738999962806702


## Stage 5: Converting the model into TensorFlow Lite version

### Saving the model

In [0]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Creating the TFLite Converter

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Converting the model

In [0]:
tflite_model = converter.convert()

### Saving the TFLite version of the model

In [0]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)